### Visualizing Filters and Feature Maps in Convolutional Neural Networks

Imports and model definitions

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import cv2 as cv
import argparse
from torchvision import models, transforms

# load the model
model = models.alexnet(pretrained=True)
print(model)
# get all the model children as list
model_children = list(model.children())

Visualizing Convolutional Layer Filters


### Exercise 1

Change `layer` value, to visualize each layer of the AlexNet. 

You should also define the y variable to visualize the convolutional filters. For this, you need to divide the shape of the first element of  `conv_layers`[`layer`].`weight` tensor, by 8 (you should also convert this to integer).


In [ ]:
# counter to keep count of the conv layers
counter = 0 
model_weights = []
conv_layers = []

for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv2d:
        counter += 1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
        for child in model_children[i].children():
            if type(child) == nn.Conv2d:
                counter += 1
                model_weights.append(child.weight)
                conv_layers.append(child)
print(f"Total convolutional layers: {counter}")

# take a look at the conv layers and the respective weights
for weight, conv in zip(model_weights, conv_layers):
    print(f"CONV: {conv} ====> SHAPE: {weight.shape}")


plt.figure(figsize=(20, 17))

# Change layer value
layer = ...
for i, filter in enumerate(model_weights[layer]):
    # Define y
    y = ...
    plt.subplot(8, y, i+1) 
    plt.imshow(filter[0, :, :].detach(), cmap='gray')
    plt.axis('off')
plt.show()

### Exercise 2
Reading an image and defining the transforms

In this exercise, you must use any image of your choice.

In [ ]:
# read and visualize an image
img = cv.imread(...)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()
# define the transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])
img = np.array(img)
# apply the transforms
img = transform(img)
print(img.size())
# unsqueeze to add a batch dimension
img = img.unsqueeze(0)
print(img.size())

Passing the Input Image Through Each Convolutional Layer


In [ ]:
# pass the image through all the layers
results = [conv_layers[0](img)]
for i in range(1, len(conv_layers)):
    # pass the result from the last layer to the next layer
    results.append(conv_layers[i](results[-1]))
# make a copy of the `results`
outputs = results

Visualizing the Feature Maps


In [ ]:
plt.figure(figsize=(30, 30))
layer_viz = outputs[layer][0, :, :, :]
layer_viz = layer_viz.data
print(layer_viz.size())
for i, filter in enumerate(layer_viz):
    plt.subplot(8, y, i + 1)
    plt.imshow(filter, cmap='gray')
    plt.axis("off")
plt.show()
plt.close()

### Directly use pre-trained AlexNet for Image Classification and Visualization of the activation maps

Imports and data transformations.

In [ ]:
import os
import torch
import torch.nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F 
import torchvision.utils as utils
import cv2 
import matplotlib.pyplot as plt
import numpy as np 
from PIL import Image
import argparse


data_transforms = transforms.Compose([
    transforms.Resize((224,224)),       # resize the input to 224x224
    transforms.ToTensor(),              # put the input to tensor format
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # normalize the input
])


Image transformations.

In [ ]:
# Use your image
img = Image.open(...)

print("original image's shape: " + str(img.size))
# pre-process the input
transformed_img = data_transforms(img)
print("transformed image's shape: " + str(transformed_img.shape))
# form a batch with only one image
batch_img = torch.unsqueeze(transformed_img, 0)
print("image batch's shape: " + str(batch_img.shape))

# load pre-trained AlexNet model
print("\nfeed the input into the pre-trained alexnet to get the output")
alexnet = models.alexnet(pretrained=True)
# put the model to eval mode for testing
alexnet.eval()

# obtain the output of the model
output = alexnet(batch_img)
print("output vector's shape: " + str(output.shape))

print(model)


### Exercise 3

Visualize activation maps.

In this exercise you need to define the variables `l1`, `l2`, `l3`, `l4` and `l5` (convolutional layer 1, convolutional layer 2, convolutional layer 3, convolutional layer 4 and convolutional layer 5). Note that these values should correspond to the index of each convolutional layer of the AlexNet (HINT: Check the layers of the model, printed in the previous cell).

In [ ]:
I = utils.make_grid(batch_img, nrow=1, normalize=True, scale_each=True)
img = I.permute((1, 2, 0)).cpu().numpy()

l1, l2 ,l3 ,l4 ,l5 = ..., ..., ..., ..., ...

conv_results = []
x = batch_img
for idx, operation in enumerate(alexnet.features):
    x = operation(x)

    if idx in {l1, l2, l3, l4, l5}:
        conv_results.append(x)

for i in range(5):
    conv_result = conv_results[i]
    N, C, H, W = conv_result.size()

    mean_acti_map = torch.mean(conv_result, 1, True)
    mean_acti_map = F.interpolate(mean_acti_map, size=[224,224], mode='bilinear', align_corners=False)

    map_grid = utils.make_grid(mean_acti_map, nrow=1, normalize=True, scale_each=True)
    map_grid = map_grid.permute((1, 2, 0)).mul(255).byte().cpu().numpy()
    map_grid = cv2.applyColorMap(map_grid, cv2.COLORMAP_JET)
    map_grid = cv2.cvtColor(map_grid, cv2.COLOR_BGR2RGB)
    map_grid = np.float32(map_grid) / 255

    visual_acti_map = 0.6 * img + 0.4 * map_grid
    tensor_visual_acti_map = torch.from_numpy(visual_acti_map).permute(2, 0, 1)

    plt.imshow(visual_acti_map)
    plt.show()